In [36]:
%pylab inline

import pandas as pd

from soln.dataset import get_augmented_train_and_test_set

%time aug_train_set, aug_test_set = get_augmented_train_and_test_set()
aug_train_set.info()

Populating the interactive namespace from numpy and matplotlib
CPU times: user 6.76 s, sys: 4 ms, total: 6.76 s
Wall time: 6.81 s
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30213 entries, 0 to 30212
Data columns (total 30 columns):
tube_assembly_id      30213 non-null object
supplier              30213 non-null object
quote_date            30213 non-null object
annual_usage          30213 non-null int64
min_order_quantity    30213 non-null int64
bracket_pricing       30213 non-null bool
quantity              30213 non-null int64
log_cost              30213 non-null float64
material_id           29984 non-null object
diameter              30213 non-null float64
wall_thickness        30213 non-null float64
length                30213 non-null float64
num_bends             30213 non-null int64
bend_radius           30213 non-null float64
end_a_1x              30213 non-null bool
end_a_2x              30213 non-null bool
end_x_1x              30213 non-null bool
end_x_2x            

In [28]:
df = aug_train_set[['tube_assembly_id', 'adj_quantity', 'bracketing_pattern']]
df[500:520]

,tube_assembly_id,adj_quantity,bracketing_pattern
500,TA-00222,5,"(1, 2, 5, 10, 25, 50, 100, 250)"
501,TA-00222,10,"(1, 2, 5, 10, 25, 50, 100, 250)"
502,TA-00222,25,"(1, 2, 5, 10, 25, 50, 100, 250)"
503,TA-00222,50,"(1, 2, 5, 10, 25, 50, 100, 250)"
504,TA-00222,100,"(1, 2, 5, 10, 25, 50, 100, 250)"
505,TA-00222,250,"(1, 2, 5, 10, 25, 50, 100, 250)"
506,TA-00228,1,"(1, 10)"
507,TA-00228,10,"(1, 10)"
508,TA-00229,5,()
509,TA-00230,1,()


In [37]:
aug_train_set.dev_fold.value_counts()

9    4695
0    3902
1    3819
2    3769
3    2959
8    2925
4    2781
5    1918
6    1856
7    1589
dtype: int64